In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
from operator import itemgetter
import numpy as np
import math
from google.colab import files

In [3]:
### Collecting Stock data of China and Visulaizing Data frame
data = pd.read_csv("drive/My Drive/CE_China.csv")
df = data.drop(data.columns[1 : 7], axis = 1)

# Exctrating data between 2013 - 2018
df = df.iloc[99 : 171,:]
print("Shape of DataFrame: ", df.shape)
df.head(5)

Shape of DataFrame:  (72, 1018)


,Name,INDUSTRIAL AND COMM - MARKET CAPITALIZATION,INDUSTRIAL AND COMM - RETURN ON EQUITY - TOTAL (%),INDUSTRIAL AND COMM - PRICE INDEX,KWEICHOW MOUTAI CO - MARKET CAPITALIZATION,KWEICHOW MOUTAI CO - RETURN ON EQUITY - TOTAL (%),KWEICHOW MOUTAI CO - PRICE INDEX,AGRICULTURAL BANK - MARKET CAPITALIZATION,AGRICULTURAL BANK - RETURN ON EQUITY - TOTAL (%),AGRICULTURAL BANK - PRICE INDEX,PETROCHINA CO LTD - MARKET CAPITALIZATION,PETROCHINA CO LTD - RETURN ON EQUITY - TOTAL (%),PETROCHINA CO LTD - PRICE INDEX,PING AN INSURANCE - MARKET CAPITALIZATION,PING AN INSURANCE - RETURN ON EQUITY - TOTAL (%),PING AN INSURANCE (GP.) CO. OF CHINA 'A' - PRICE INDEX,BANK OF CHINA LTD - MARKET CAPITALIZATION,BANK OF CHINA LTD - RETURN ON EQUITY - TOTAL (%),BANK OF CHINA LTD - PRICE INDEX,CHINA MERCHANTS BANK - MARKET CAPITALIZATION,CHINA MERCHANTS BANK - RETURN ON EQUITY - TOTAL (%),CHINA MERCHANTS BANK - PRICE INDEX,CHINA LIFE INSURANCE - MARKET CAPITALIZATION,CHINA LIFE INSURANCE - RETURN ON EQUITY - TOTAL (%),CHINA LIFE INSURANCE - PRICE INDEX,CHINA PETROLEUM - MARKET CAPITALIZATION,CHINA PETROLEUM - RETURN ON EQUITY - TOTAL (%),CHINA PETROLEUM - PRICE INDEX,CHINA YANGTZE POWER - MARKET CAPITALIZATION,CHINA YANGTZE POWER - RETURN ON EQUITY - TOTAL (%),CHINA YANGTZE POWER - PRICE INDEX,INDUSTRIAL BANK - MARKET CAPITALIZATION,INDUSTRIAL BANK - RETURN ON EQUITY - TOTAL (%),INDUSTRIAL BANK - PRICE INDEX,JIANGSU HENGRUI MED - MARKET CAPITALIZATION,JIANGSU HENGRUI MED - RETURN ON EQUITY - TOTAL (%),JIANGSU HENGRUI MED - PRICE INDEX,SHANGHAI PUDONG - MARKET CAPITALIZATION,SHANGHAI PUDONG - RETURN ON EQUITY - TOTAL (%),SHANGHAI PUDONG - PRICE INDEX,...,BESTTONE HOLDING - PRICE INDEX,BETHEL AUTOMOTIVE - MARKET CAPITALIZATION,BETHEL AUTOMOTIVE - RETURN ON EQUITY - TOTAL (%),BETHEL AUTOMOTIVE - PRICE INDEX,BLACK PEONY - MARKET CAPITALIZATION,BLACK PEONY - RETURN ON EQUITY - TOTAL (%),BLACK PEONY - PRICE INDEX,BOMIN ELECTRO - MARKET CAPITALIZATION,BOMIN ELECTRO - RETURN ON EQUITY - TOTAL (%),BOMIN ELECTRO - PRICE INDEX,BRIGHT DAIRY AN - MARKET CAPITALIZATION,BRIGHT DAIRY AN - RETURN ON EQUITY - TOTAL (%),BRIGHT DAIRY AN - PRICE INDEX,BROS EASTERN - MARKET CAPITALIZATION,BROS EASTERN - RETURN ON EQUITY - TOTAL (%),BROS EASTERN - PRICE INDEX,BTG HOTELS - MARKET CAPITALIZATION,BTG HOTELS - RETURN ON EQUITY - TOTAL (%),BTG HOTELS - PRICE INDEX,CAMEL GROUP CO LTD - MARKET CAPITALIZATION,CAMEL GROUP CO LTD - RETURN ON EQUITY - TOTAL (%),CAMEL GROUP CO LTD - PRICE INDEX,CCS SUPPLY CHAIN - MARKET CAPITALIZATION,CCS SUPPLY CHAIN - RETURN ON EQUITY - TOTAL (%),CCS SUPPLY CHAIN - PRICE INDEX,CECEP WIND-POWER - MARKET CAPITALIZATION,CECEP WIND-POWER - RETURN ON EQUITY - TOTAL (%),CECEP WIND-POWER - PRICE INDEX,CENTER INTERNATIONAL - MARKET CAPITALIZATION,CENTER INTERNATIONAL - RETURN ON EQUITY - TOTAL (%),CENTER INTERNATIONAL - PRICE INDEX,CH JINGGONG STEEL - MARKET CAPITALIZATION,CH JINGGONG STEEL - RETURN ON EQUITY - TOTAL (%),CH JINGGONG STEEL - PRICE INDEX,CHANGJIANG PUB - MARKET CAPITALIZATION,CHANGJIANG PUB - RETURN ON EQUITY - TOTAL (%),CHANGJIANG PUB - PRICE INDEX,CHANGSHU FENGFAN - MARKET CAPITALIZATION,CHANGSHU FENGFAN - RETURN ON EQUITY - TOTAL (%),CHANGSHU FENGFAN - PRICE INDEX
99,01/28/2013,1.257971e+09,21.91,133.1,133281543.0,39.43,2085.1,805489410.0,20.88,107.8,1.411092e+09,11.8,20.6,330340609.0,16.21,99.7,731908587.0,17.95,82.3,274644119.0,22.22,356.5,427644987.0,10.95,52.0,522212606.0,12.25,160.6,104280000.0,11.85,186.7,193190695.0,22.32,170.3,51661189.0,21.4,1110.5,175902353.0,21.43,185.9,...,91.2,NaN,NaN,NaN,4844733.0,8.95,118.7,NaN,16.81,NaN,27183844.0,9.79,127.6,7597500.0,8.45,67.0,3336788.0,11.36,53.2,9283680.0,15.99,115.9,8329738.0,31.51,107.5,NaN,7.18,NaN,NaN,NaN,NaN,4105968.0,10.76,226.1,9187333.0,10.23,184.1,5861945.0,8.06,61.0
100,02/28/2013,1.257971e+09,21.91,130.6,133281543.0,39.43,2044.2,805489410.0,20.88,111.1,1.411092e+09,11.8,20.6,330340609.0,16.21,100.6,731908587.0,17.95,82.0,274644119.0,22.22,334.4,427644987.0,10.95,48.5,522212606.0,12

In [4]:
### utility functions

def avg(list):
    list = np.array(list)
    return np.sum(list)/len(list)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def isNaN(x):
    return str(float(x)).lower() == 'nan'

In [5]:
def get_smb_hml(df, row):
    """
    Returns SMB and HML of the given month represented by the row
    """

    total = 0
    cnt = 0
    m_cap_list = []
    roe_list = []
    return_list = []
    idx = 0

    z = df.iloc[row] 
    z1 = df.iloc[row - 1]

    for i in range(int((len(df.columns) - 1)/3)):
        total = total + 1 
        m_cap = z[3*i + 1]
        roe = z[3*i + 2]
        price_t = z[3*i + 3]
        price_t_1 = z1[3*i + 3]

        if not isNaN(m_cap) and not isNaN(roe) and not isNaN(price_t) and not isNaN(price_t_1):
            m_cap_list.append((m_cap,idx))
            roe_list.append((roe,idx))
            ret=(math.log(price_t/price_t_1))*100
            return_list.append(ret)
            idx = idx + 1
            cnt = cnt + 1

    m_cap_list = sorted(m_cap_list, key = itemgetter(0))
    roe_list = sorted(roe_list, key = itemgetter(0))

    ### Dividing companies in SMALL/BIG and VALUE/NEUTRAL/GROWTH
    small = m_cap_list[0 : int(len(m_cap_list)/2)]
    small_i = [b for (a, b) in small]
    big = m_cap_list[int(len(m_cap_list)/2) : int(len(m_cap_list))]
    big_i = [b for (a, b) in big]

    growth = roe_list[0 : int(len(roe_list)*0.3)]
    growth_i = [b for (a, b) in growth]
    neutral = roe_list[int(len(roe_list)*0.3) : int(len(roe_list)*0.7)]
    neutral_i = [b for (a, b) in neutral]
    value = roe_list[int(len(roe_list)*0.7) : int(len(roe_list))]
    value_i = [b for (a, b) in value]

    ### Created 6 portfolios: SV, SN, SG and BN, BG, BV
    sv = intersection(small_i, value_i)
    sn = intersection(small_i, neutral_i)
    sg = intersection(small_i, growth_i)
    bv = intersection(big_i, value_i)
    bn = intersection(big_i, neutral_i)
    bg = intersection(big_i, growth_i)

    sv_return_list = [return_list[i] for i in sv]
    sn_return_list = [return_list[i] for i in sn]
    sg_return_list = [return_list[i] for i in sg]

    bv_return_list = [return_list[i] for i in bv]
    bn_return_list = [return_list[i] for i in bn]
    bg_return_list = [return_list[i] for i in bg]

    ### Calculating average return of the Portfolios
    sv_return = avg(sv_return_list)
    sn_return = avg(sn_return_list)
    sg_return = avg(sg_return_list)
    bv_return = avg(bv_return_list)
    bn_return = avg(bn_return_list)
    bg_return = avg(bg_return_list)

    sml = (1/3)*((sv_return + sn_return + sg_return) - (bv_return + bn_return + bg_return))
    hml = 0.5*(sv_return + bv_return) - 0.5*(sg_return + bg_return)  
    return [sml, hml]

In [6]:
data = []
### Calculating SML and HML and Return_of_INDUSTRIAL_and_COMM for all months
for i in range(1, 72):
    x = get_smb_hml(df, i)
    x.insert(0, df.iloc[i, 0])
    price_t = df.iloc[i,3]
    price_t_1 = df.iloc[i - 1,3]
    ret = (math.log(price_t/price_t_1))*100
    x.append(ret)
    data.append(x)
  
df1 = pd.DataFrame(data, columns = ['Date', 'SMB','HML','Return_of_INDUSTRIAL AND COMM']) 
print(df1.head(10))

df1.to_csv('regression_data.csv')
# files.download('regression_data.csv')

         Date       SMB       HML  Return_of_INDUSTRIAL AND COMM
0  02/28/2013  1.278202  1.311712                      -1.896151
1  03/28/2013  0.617475  4.883151                      -4.863704
2  04/28/2013 -1.459820  3.404877                       1.039594
3  05/28/2013  4.903316  0.784893                       3.363633
4  06/28/2013 -1.297867  9.406113                      -4.162360
5  07/28/2013  4.332276  0.603131                      -2.764404
6  08/28/2013  4.945289  1.072659                      -0.744728
7  09/28/2013 -1.451600 -1.436405                      -0.499585
8  10/28/2013  1.419685  3.120159                      -2.621715
9  11/28/2013 -1.131754 -3.850613                       1.023027


###Carhart Model
We also calculate the 4th factor -  **Momentum Factor** 

In [7]:
def get_momentum_factor(df, row):
    """
    Returns Momentum Factor of the given month represented by the row
    """

    total = 0
    cnt = 0
    m_cap_list = []
    return_list = []
    return11_list = []
    idx = 0

    z = df.iloc[row] 
    z1 = df.iloc[row - 1]
    tm1 = df.iloc[row - 1]
    tm12 = df.iloc[row - 12]

    for i in range(int((len(df.columns) - 1)/3)):
        total = total + 1 
        m_cap = z[3*i + 1]
        price_t = z[3*i + 3]
        price_t_1 = z1[3*i + 3]
        price_tm1 = tm1[3*i + 3]
        price_tm12 = tm12[3*i + 3]

        if not isNaN(m_cap) and not isNaN(price_tm1) and not isNaN(price_tm12) and not isNaN(price_t) and not isNaN(price_t_1):
            m_cap_list.append((m_cap, idx))
            ret = (math.log(price_t/price_t_1))*100
            return_list.append((ret,idx))
            ret_11 = (math.log(price_tm1/price_tm12))*100
            return11_list.append((ret_11,idx))
            idx = idx + 1
            cnt = cnt + 1

    m_cap_list = sorted(m_cap_list, key = itemgetter(0))
    momentum_return = sorted(return11_list, key = itemgetter(0))

    ### Finding Winner and Loser portfolios
    loser = momentum_return[0 : int(len(momentum_return)*0.3)]
    winner = momentum_return[int(len(momentum_return)*0.7) : int(len(momentum_return))]

    small = m_cap_list[0 : int(len(m_cap_list)/2)]
    small_i = [b for (a, b) in small]
    big = m_cap_list[int(len(m_cap_list)/2) : int(len(m_cap_list))]
    big_i = [b for (a, b) in big]

    winner_index = [b for (a, b) in winner]
    loser_index = [b for (a, b) in loser]

    ### Forming Winner - big, Winner - small, Loser - big, Loser - small portfolios
    WB = intersection(big_i, winner_index)
    WS = intersection(small_i, winner_index)
    LB = intersection(big_i, loser_index)
    LS = intersection(small_i, loser_index)


    ### Finding equally weigted returns
    WS_VALUE = [a for (a, b) in return_list if b in WS]
    WB_VALUE = [a for (a, b) in return_list if b in WB]
    LS_VALUE = [a for (a, b) in return_list if b in LS]
    LB_VALUE = [a for (a, b) in return_list if b in LB]

    WS_VALUE = avg(WS_VALUE)
    WB_VALUE = avg(WB_VALUE)
    LS_VALUE = avg(LS_VALUE)
    LB_VALUE = avg(LB_VALUE)

    return (WS_VALUE - LS_VALUE + WB_VALUE - LB_VALUE)/2;

In [8]:
data = []

for i in range(13, 72):
    x = [get_momentum_factor(df, i)]
    x.insert(0, df.iloc[i, 0])
    data.append(x)

df2 = pd.DataFrame(data, columns = ['Date', 'Momentum_Factor'])
print(df2.head(10))

df2.to_csv('momentum_factor_data.csv')
# files.download('momentum_factor_data.csv')

         Date  Momentum_Factor
0  02/28/2014         2.554836
1  03/28/2014        -5.608138
2  04/28/2014        -1.955867
3  05/28/2014         0.727727
4  06/28/2014         1.842021
5  07/28/2014        -5.157732
6  08/28/2014         0.147788
7  09/28/2014        -0.021563
8  10/28/2014         1.491300
9  11/28/2014        -5.076985
